In [128]:
import math
from collections import Counter
import pickle
import re
import nltk
from nltk.corpus import stopwords
from lyricsgenius import Genius

with open('../lyrics-updated.pkl', 'rb') as f:
    song_lyrics = pickle.load(f)


with open('../classifications.pkl', 'rb') as f1:
    classifications = pickle.load(f1)

with open('../output/big-songs-updated.pkl', 'rb') as f2:
    big_songs = pickle.load(f2)

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
def tokenize(lyrics):
    words = lyrics.lower().split()
    return [word for word in words if word not in stop_words]

k1 = 1.0
b = 0.9

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\civip\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [129]:
# call the api to get the lyrics

genius = Genius('HhcPCGMwENzdfMQlauEV1Om0UormkWc6Wrwxri-LFTfBSNkzhwMSVUnR5tZG8eXW')
genius.remove_section_headers = True
genius.skip_non_songs = False
genius.excluded_terms = ["Remix", "Live"]

# song_name = "Happy"
# artist = "Pharrel Williams"
# song_name = "Firework"
# artist = "Katy Perry"
# song_name = "I Hate Everything About You"
# artist = "Three Days Grace"
# song_name = "Coffee"
# artist = "Chappell Roan"
song_name = "Tears in Heaven"
artist = "Eric Clapton"


song = genius.search_song(song_name, artist)
pos = song.lyrics.find(f'{song_name} Lyrics')
query = song.lyrics[pos:]



Searching for "Tears in Heaven" by Eric Clapton...
Done.


In [130]:
# then remove  capital letters and non alphanumeric characters
query = query.lower()
query = ''.join([char if char.isalpha() or char.isspace() else '' for char in query.replace('\n', ' ')])
# it might seem crazy what im bout to say
query



'tears in heaven lyrics would you know my name if i saw you in heaven would it be the same if i saw you in heaven  i must be strong and carry on cause i know i dont belong here in heaven  would you hold my hand if i saw you in heaven would you help me stand if i saw you in heaven  ill find my way through night and day cause i know i just cant stay here in heaven  time can bring you down time can bend your knees time can break your heart have you begging please begging please you might also like  beyond the door theres peace im sure and i know therell be no more tears in heaven  would you know my name if i saw you in heaven would you be the same if i saw you in heaven  i must be strong and carry on cause i know i dont belong here in heavenembed'

In [131]:
# tokenize all documents
tokenized_songs = {}
for category, all_lyrics in big_songs.items():
    tokenized_songs[category] = tokenize(all_lyrics)

# calculate document lengths
doc_lengths = {}
for  category, all_lyrics in tokenized_songs.items():
    doc_lengths[category] = len(all_lyrics)

# avg doc length
avg_doc_length = sum(doc_lengths.values()) / len(doc_lengths)


In [132]:
# calculate doc freqs
doc_freqs = {}
for doc in tokenized_songs.values():
    unique_terms = set(doc)
    for term in unique_terms:
        doc_freqs[term] = doc_freqs.get(term, 0) + 1

In [133]:
# bm25 algorithm
def bm25(query, document, doc_length):
    score = 0.0
    query_terms = tokenize(query)
    term_freqs = Counter(document)
    
    for term in query_terms:
        if term in doc_freqs:  # term must appear in the corpus
            # Calculate BM25 components
            doc_freq = doc_freqs[term]
            idf = math.log((len(big_songs) - doc_freq + 0.5) / (doc_freq + 0.5) + 1)
            term_freq = term_freqs[term]
            term_score = idf * (term_freq * (k1 + 1)) / (term_freq + k1 * (1 - b + b * doc_length / avg_doc_length))
            score += term_score
    return score

In [134]:
scores = []
for doc_id, doc in tokenized_songs.items():
    score = bm25(query, doc, doc_lengths[doc_id])
    scores.append((doc_id, score))

# Sort documents by score in descending order
ranked_docs = sorted(scores, key=lambda x: x[1], reverse=True)

# Display the ranking results
print("Document Rankings:")
for rank, (doc_id, score) in enumerate(ranked_docs, start=1):
    print(f"Rank {rank}: Document {doc_id} with score {score:.4f}")

Document Rankings:
Rank 1: Document mild with score 9.0468
Rank 2: Document anxious with score 9.0352
Rank 3: Document eager with score 8.9053
Rank 4: Document lukewarm with score 8.5406
Rank 5: Document thrilling with score 7.3612
Rank 6: Document excited with score 7.2348
Rank 7: Document curious with score 7.0540
Rank 8: Document confident with score 7.0420
Rank 9: Document electric with score 6.9851
Rank 10: Document neutral with score 6.7555
Rank 11: Document passionate with score 6.6376
Rank 12: Document indifferent with score 6.6092
Rank 13: Document enthusiastic with score 6.2190
Rank 14: Document animated with score 6.2130
Rank 15: Document hectic with score 5.7224
Rank 16: Document delightful with score 5.5865
Rank 17: Document pleasing with score 5.4591
Rank 18: Document fulfilled with score 5.3099
Rank 19: Document happy with score 5.2761
Rank 20: Document frustration with score 5.2156
Rank 21: Document distressed with score 4.9994
Rank 22: Document content with score 4.569